In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from datetime import date
from datetime import timedelta
from fractions import Fraction
import requests
import cloudscraper
import pandas as pd
import math
import re

def scrape(url_1_1,url_1_2):
    today = date.today()
    str_today = str(today)

    def find_between( s, first, last ):
        try:
            start = s.index( first ) + len( first )
            end = s.index( last, start )
            return s[start:end]
        except ValueError:
            return ""

    def getPlayer_nones_1(id):
        data = soup_1_1.find(id=id)
        data1 = str(data.find(class_="panel-heading"))
        data2 = find_between(data1, "<b>", "</b>")
        data3 = data2.replace(", ","")
        return data3

    def getPlayer_nones_2(id):
        data = soup_1_2.find(id=id)
        data1 = str(data.find(class_="panel-heading"))
        data2 = find_between(data1, "<b>", "</b>")
        data3 = data2.replace(", ","")
        return data3

    def getPlayer_1(id):
        data = soup_1_1.find(id=id)
        data1 = data.find(class_="panel-heading")
        data2 = str(data1.find(class_="report-link"))
        data3 = re.split('Scouting Report.">',data2,3)[-1]
        data4 = data3.replace("</a>","")
        return data4

    def getPlayer_2(id):
        data = soup_1_2.find(id=id)
        data1 = data.find(class_="panel-heading")
        data2 = str(data1.find(class_="report-link"))
        data3 = re.split('Scouting Report.">',data2,3)[-1]
        data4 = data3.replace("</a>","")
        return data4

    #url_1_1 = "https://walterfootball.com/draft2022charlie.php"
    #url_1_2 = "https://walterfootball.com/draft2022charlie_1.php"
    r_1_1 = requests.get(url_1_1)
    r_1_2 = requests.get(url_1_2)


    # Convert to bs object
    soup_1_1 = bs(r_1_1.text,'lxml')
    contents_1_1 = soup_1_1.prettify()

    soup_1_2 = bs(r_1_2.text,'lxml')
    contents_1_2 = soup_1_2.prettify()

    col_list = ['mockDraftSlot_' + str(n) for n in range(65)]

    draft_1_1 = []
    draft_1_2 = []
    draft_1_1_nones = []
    draft_1_2_nones = []


    i = 1
    while i<17:
        draft_1_1.append(getPlayer_1(col_list[i]))
        i+=1

    i = 17
    while i<33:
        draft_1_2.append(getPlayer_2(col_list[i]))
        i+=1

    # Combining draft_1_1 and draft_1_2
    draft = list(draft_1_1+draft_1_2)

    i = 1
    while i<17:
        draft_1_1_nones.append(getPlayer_nones_1(col_list[i]))
        i+=1

    i = 17
    while i<33:
        draft_1_2_nones.append(getPlayer_nones_2(col_list[i]))
        i+=1

    # Combining draft_1_1_nones and draft_1_2_nones
    draft_nones = list(draft_1_1_nones+draft_1_2_nones)

    # Iterating through draft and adding draft_none element if draft element is = none
    for i, n in enumerate(draft):
        if n == "None":
            draft[i] = draft_nones[i]

    df_draft = pd.DataFrame(data=draft)
    df_draft.index +=1
    df_draft.columns = ['player']
    df_draft.to_excel("WalterDraft---"+str_today+".xlsx", header=None,index=False)

In [2]:
scrape("https://walterfootball.com/draft2022charlie.php", "https://walterfootball.com/draft2022charlie_1.php")